# Базы данных
![](https://i.ibb.co/8nkyKwT/db-meme.jpg)
База данных - это технология, с помощью которой удобно хранить данные. У нее есть множество реализаций.

Но основная идея состоит в следующем:
* есть несколько зафиксированных **таблиц**
* у каждой таблицы зафиксировано, какие в ней есть **столбцы** / поля
* столбец определяется его названием и типом данных, который в нем лежит
* основном объект хранения в базе данных - это **строчки**
* каждая строчка соответствует какому-то объекту, и содержит значение каждого поля
* есть **операции** создания таблицы, добавления/удаления/изменения строк в таблицах
* есть язык для описания почти сколь угодно умных **запросов** к базе данных, он называется **SQL**
* таблицы обычно глубоко взаимосвязаны: в столбцах разных таблиц могут храниться одни и те же id-шники, например

Пример архитектуры несложной базы данных:

* Таблица Students, состоящая из 3 столбцов StudentId (целое число), StudentName (строка) и BirthDate (дата)
* Таблица Classes, состоящая из 2 столбцов ClassId (целое число) и ClassName (строка)
* Таблица StudentClass, состоящая из 2 столбцов StudentId (целое число), ClassId (целоечисло) - она содер

![Пример базы данных](https://i.ibb.co/zStny22/tables.jpg)

Могут возникнуть вопросы: правда ли тут нужны 3 таблицы? И зачем нам нужны Id, если можно писать просто имена? Подумайте, как ответить на эти вопросы.

Какие есть свойства у базы данных, что люди используют ее вместо pandas.DataFrame или Excel?
* в базе данных удобно хранить сразу несколько таблиц
* у базы данных есть очень гибкий язык запросов SQL
* база данных запущена локально на сервере и обновляется с помощью запросов к ней
* обычно базы отвечают на запросы очень быстро, потому что хранят данные не втупую подряд, а делают индексы по разным столбцам (это кстати можно контролировать)
* база данных умеет обновляться атомарно: можно сделать несколько действий (запрос + удаление строчки + добавление строчки + еще что-нибудь) внутри одной транзакции, и не будет такого момента, когда половина запроса выполнилась, а другая - нет

Нужно понимать, что база данных - это не единая технология, это набор разных идей, которы содержатся в разных реализациях в разных комбинациях. Например, базу данных можно поднять на распределенной системе кластера серверов и писать для нее запросы как в SQL, почти не задумываясь о том, что содержимое одной таблицы может лежать на 5 разных серверах.

Если вы делаете серьезный проект: сайт, приложение, да даже бота - вам скорее всего понадобится база данных. Это точно лучше по памяти и времени, чем хранить все в оперативной памяти или тупо в файлике (если у вас хоть сколько-то много объектов - больше тысячи или десяти тысяч). Она будет запущена на сервере рядом с вашим основным скриптом, который будет иногда делать к ней запросы.

## Установка базы и библиотеки

Как я уже говорил, реализаций баз данных много, давай например возьмем PostgreSQL.

Сначала поставим ее на компьютер

### на Linux или MacOS

In [9]:
# Запустите из консоли вот эту строчку
# sudo apt-get install postgresql postgresql-contrib

### на Windows

In [ ]:
# TO BE DISCOVERED

### дальше одинаково из консоли

Проверим, что все хорошо, и сервис включился:

In [12]:
!/etc/init.d/postgresql status

● postgresql.service - PostgreSQL RDBMS
   Loaded: loaded (/lib/systemd/system/postgresql.service; enabled; vendor preset: enabled)
   Active: active (exited) since Tue 2019-07-23 20:59:45 MSK; 13min ago
 Main PID: 4744 (code=exited, status=0/SUCCESS)
    Tasks: 0 (limit: 4915)
   CGroup: /system.slice/postgresql.service


Если сервис не включен, попробуйте его включить вот такими операциями:

In [ ]:
# sudo service postgresql start
# sudo service postgresql restart

Давайте создадим себе пользователя и тестовую базу данных.

При создании пользователя вас также попросят создать пароль.

In [17]:
# sudo -u postgres createuser -D -A -P username
# sudo -u postgres createdb -O username testdb

/bin/sh: 1: postgres: not found
/bin/sh: 1: postgres: not found


Перегрузим сервис, чтобы база создалась.

In [131]:
! /etc/init.d/postgresql reload

Reloading postgresql configuration (via systemctl): postgresql.serviceFailed to reload postgresql.service: Access denied
See system logs and 'systemctl status postgresql.service' for details.
 failed!


Поставьте питоновскую библиотеку для рабоыт с этой базой из питона.

In [4]:
!pip3 install psycopg2-binary

    100% |████████████████████████████████| 2.9MB 662kB/s eta 0:00:011


## Как работать с базой из питона

Воспользуемся библиотекой psycopg2 для работы с PostgreSQL. Это не самая приятная библиотека, но нам нужно оттуда буквально пара штук.

In [140]:
import psycopg2

con = psycopg2.connect(dbname='testdb', user='username', 
                           password='password', host='localhost')
cur = con.cursor()
cur.execute('''
DROP TABLE IF EXISTS Students;
CREATE TABLE Students(StudentId INT PRIMARY KEY, Name VARCHAR(30), BirthDate DATE);
INSERT INTO Students VALUES(1, 'Николай Иванов', '2003.03.01');
INSERT INTO Students VALUES(2, 'Иван Даниилов', '2002.06.10');
INSERT INTO Students VALUES(3, 'Даниил Дмитриев', '2002.05.28');
INSERT INTO Students VALUES(4, 'Дмитрий Николаев', '2003.12.01');
''')
con.commit()

Можете попробовать выполнить запрос еще раз: увидите, что у вас ничего не выйдет.

Давайте сделаем функцию, чтобы не писать каждый раз connect и commit. А еще обернем функцию в try-except.

In [141]:
def make_sql(query):
    try:
        con = psycopg2.connect(dbname='testdb', user='username', 
                               password='password', host='localhost')
        cur = con.cursor()
        cur.execute(query) # вот это самая главная строчка - выполнить запрос
        con.commit()
        return True
    except psycopg2.DatabaseError as e:
        if con:
            con.rollback() # откатить изменения всех действий, если хоть в каком-то месте ошибка
        print('Error in DataBase: {}'.format(e))
        return False

In [142]:
make_sql('CREATE TABLE IF NOT EXISTS Students(StudentId INT PRIMARY KEY, Name VARCHAR(30), BirthDate DATE)')

True

In [154]:
make_sql('CREATE TABLE Students(StudentId INT PRIMARY KEY, Name VARCHAR(30), BirthDate DATE)')

Error in DataBase: relation "students" already exists



False

## Язык SQL
SQL = Structured Query Language = язык структурированных запросов.

Язык придумали еще в 80-ые, и его придумывали изначально простым и понятным для любых пользователей. Общая структура запросов действительно очень простая и читаемая, но из-за этого из структуры зарпоса абсолютно нетривиально понять, как именно база данных будет его выполнять. Но для пользователя обычно это и не очень важно.

Запросы в SQL иногда излишне длинные и имеют повторяющиеся части. Очень часто запрос на языке SQL составляется внутри другого языка программирования (питон) как длинная строка, куда подставляются какие-то ключевые слова, и потом уже передается базе данных. Это нормальная практика, не стесняйтесь так делать.

### Создание и удаление таблицы
Функции `CREATE TABLE` и `DROP TABLE` вы уже видели:

In [147]:
make_sql('DROP TABLE Students')

Error in DataBase: table "students" does not exist



False

In [148]:
make_sql('DROP TABLE Students')

Error in DataBase: table "students" does not exist



False

In [163]:
make_sql('DROP TABLE IF EXISTS Students') # рекомендую простой if на существование таблицы, чтобы не было ошибок

True

In [164]:
make_sql('CREATE TABLE IF NOT EXISTS Students(StudentId INT PRIMARY KEY, Name TEXT, BirthDate DATE)')

True

В скобках после названия таблицы нужно описать все столбцы этой таблицы: название и тип столбца.

**Некоторые типы данных:**

Целые числа
* INT - целое 32-битное число
* BIGINT - целое 64-битное число
* BOOLEAN - бит

Вещественные числа
* REAL - вещественное 32-битное число с плавающей точкой
* DOUBLE PRECISION - вещественное 64-битное число с плавающей точкой

Строки
* VARCHAR(N) - строка длины до N символов
* TEXT - строка переменной длины (менее оптимально по памяти)

Разное всякое
* DATE - дата
* TIME - время
* TIMESTAMP - дата и время

На самом деле типов в разы больше, всегда можете загуглить. Не все из них работают для всех реализаций баз данных.

### Вставка в таблицу

Вставить одну строку очень просто с помощью `INSERT`. Если вам нужны более экзотические вещи - гуглите.

In [165]:
make_sql("INSERT INTO Students VALUES(1, 'Николай Иванов', '2003.03.01');")
make_sql("INSERT INTO Students VALUES(2, 'Иван Даниилов', '2002.06.10');")
make_sql("INSERT INTO Students VALUES(3, 'Даниил Дмитриев', '2002.05.28');")
make_sql("INSERT INTO Students VALUES(4, 'Дмитрий Николаев', '2003.12.01');")

True

### Запросы к таблице

А это самое интересное - запрос `SELECT`. В отличие от прошлых операций эта что-то возвращает (таблицу с результатом запроса).

Давайте напишем для нее поэтому свою функцию.

In [166]:
def select_sql(query):
    try:
        con = psycopg2.connect(dbname='testdb', user='username', 
                               password='password', host='localhost')
        cur = con.cursor()
        cur.execute(query) # главная функция
        con.commit()

        # вытаскивание результата запроса с помощью cur.fetchone()
        rows = []
        while True:
            row = cur.fetchone()
            if row == None:
                break
            rows.append(row)
        return rows
    except psycopg2.DatabaseError as e:
        if con:
            con.rollback() # откатить изменения всех действий, если хоть в каком-то месте ошибка
        print('Error in DataBase: {}'.format(e))
        return False

Теперь давайте изучать разные фишечки.

* Простейшие запросы

In [168]:
select_sql('SELECT Name FROM Students') # вернуть столбец Name

[('Николай Иванов',),
 ('Иван Даниилов',),
 ('Даниил Дмитриев',),
 ('Дмитрий Николаев',)]

In [169]:
select_sql('SELECT Name, BirthDate FROM Students')

[('Николай Иванов', datetime.date(2003, 3, 1)),
 ('Иван Даниилов', datetime.date(2002, 6, 10)),
 ('Даниил Дмитриев', datetime.date(2002, 5, 28)),
 ('Дмитрий Николаев', datetime.date(2003, 12, 1))]

* Знак `*` означает "все столбцы"

In [170]:
select_sql('SELECT * FROM Students') # вернуть всё из таблицы Students

[(1, 'Николай Иванов', datetime.date(2003, 3, 1)),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10)),
 (3, 'Даниил Дмитриев', datetime.date(2002, 5, 28)),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1))]

* `WHERE` - это IF. Так можно писать фильтры. Доступны булевы функции типа `AND`, `OR`, `NOT`, `<`, `<=`, `=`.

In [178]:
select_sql('SELECT * FROM Students WHERE StudentId > 2')

[(3, 'Даниил Дмитриев', datetime.date(2002, 5, 28)),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1))]

In [179]:
select_sql("SELECT * FROM Students WHERE BirthDate > '2003.01.01'")

[(1, 'Николай Иванов', datetime.date(2003, 3, 1)),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1))]

In [180]:
select_sql("SELECT * FROM Students WHERE StudentId > 2 AND BirthDate > '2003.01.01'")

[(4, 'Дмитрий Николаев', datetime.date(2003, 12, 1))]

In [181]:
select_sql("SELECT * FROM Students WHERE StudentId = 3")

[(3, 'Даниил Дмитриев', datetime.date(2002, 5, 28))]

* `ORDER BY` - это сортировка по столбцу (или нескольким столбцам). Можно и по убыванию.

In [182]:
select_sql('SELECT * FROM Students ORDER BY BirthDate')

[(3, 'Даниил Дмитриев', datetime.date(2002, 5, 28)),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10)),
 (1, 'Николай Иванов', datetime.date(2003, 3, 1)),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1))]

In [183]:
select_sql('SELECT * FROM Students ORDER BY Name')

[(3, 'Даниил Дмитриев', datetime.date(2002, 5, 28)),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1)),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10)),
 (1, 'Николай Иванов', datetime.date(2003, 3, 1))]

In [185]:
select_sql('SELECT * FROM Students ORDER BY Name DESC')

[(1, 'Николай Иванов', datetime.date(2003, 3, 1)),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10)),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1)),
 (3, 'Даниил Дмитриев', datetime.date(2002, 5, 28))]

### Несколько таблиц
Давайте создадим еще две таблицы и посмотрим, как выглядят более сложные операции.

In [187]:
make_sql('DROP TABLE IF EXISTS Classes;')
make_sql('CREATE TABLE Classes(ClassId INT PRIMARY KEY, Name TEXT);')
make_sql("INSERT INTO Classes VALUES(1, 'Математика');")
make_sql("INSERT INTO Classes VALUES(2, 'ML');")
make_sql("INSERT INTO Classes VALUES(3, 'Алгоритмы');")

True

In [188]:
select_sql('SELECT * FROM Classes')

[(1, 'Математика'), (2, 'ML'), (3, 'Алгоритмы')]

In [189]:
make_sql('DROP TABLE IF EXISTS StudentClass;')
make_sql('CREATE TABLE StudentClass(StudentId INT, ClassId INT);')
make_sql("INSERT INTO StudentClass VALUES(1, 2);")
make_sql("INSERT INTO StudentClass VALUES(2, 1);")
make_sql("INSERT INTO StudentClass VALUES(2, 3);")
make_sql("INSERT INTO StudentClass VALUES(3, 1);")
make_sql("INSERT INTO StudentClass VALUES(3, 2);")
make_sql("INSERT INTO StudentClass VALUES(4, 2);")

True

In [190]:
select_sql('SELECT * FROM StudentClass')

[(1, 2), (2, 1), (2, 3), (3, 1), (3, 2), (4, 2)]

* `t1 JOIN t2 ON x` - операция объединения таблиц `t1` и `t2` по условию `x`. Это значит, что для всех пар строк, где первая взята из первой таблицы, а вторая - из второй, у которых выполняется условие `x`, мы кладем в результат новую строку, где лежат столбцы и из первой таблицы, и из второй.

In [193]:
select_sql('SELECT * FROM Students JOIN StudentClass ON Students.StudentId = StudentClass.StudentId')

[(1, 'Николай Иванов', datetime.date(2003, 3, 1), 1, 2),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10), 2, 1),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10), 2, 3),
 (3, 'Даниил Дмитриев', datetime.date(2002, 5, 28), 3, 1),
 (3, 'Даниил Дмитриев', datetime.date(2002, 5, 28), 3, 2),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1), 4, 2)]

Часто нужно сделать много `JOIN`-ов.

In [197]:
select_sql('''
SELECT * FROM
    (Students
     JOIN
     StudentClass
     ON Students.StudentId = StudentClass.StudentId)
    JOIN
    Classes
    ON StudentClass.ClassId = Classes.ClassId
''')

[(1, 'Николай Иванов', datetime.date(2003, 3, 1), 1, 2, 2, 'ML'),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10), 2, 1, 1, 'Математика'),
 (2, 'Иван Даниилов', datetime.date(2002, 6, 10), 2, 3, 3, 'Алгоритмы'),
 (3, 'Даниил Дмитриев', datetime.date(2002, 5, 28), 3, 1, 1, 'Математика'),
 (3, 'Даниил Дмитриев', datetime.date(2002, 5, 28), 3, 2, 2, 'ML'),
 (4, 'Дмитрий Николаев', datetime.date(2003, 12, 1), 4, 2, 2, 'ML')]

Это довольно затратная операция, зато с ее помощью можно реализовать много разных естественных запросов.

Например, **найти все классы, на которые ходит Иван Даниилов**.

In [199]:
select_sql('''
SELECT Classes.Name FROM
    (Students
     JOIN
     StudentClass
     ON Students.StudentId = StudentClass.StudentId)
    JOIN
    Classes
    ON StudentClass.ClassId = Classes.ClassId
    WHERE Students.Name = 'Иван Даниилов'
''')

[('Математика',), ('Алгоритмы',)]

Попробуйте сами реализовать несколько запросов:

**Найти имена всех школьников, которые ходят на ML**.

**Найти всех школьников, которые ходят в один класс с Иваном Данииловым**.

* `GROUP BY key` - сгруппировать строки по столбцу `key`. Это особая магия, которая превращает запрос в совсем непривычную вещь. Если использовать группировку, то в сам `SELECT` нельзя просто так писать названия столбцов - это не имеет смысла. Туда надо писать функции агреггирования значений столбцов - например, `COUNT` или `MAX`.

Для каждого разного значения столбца `key` найдутся все строчки с этим значением, и по ним посчитается эта агрегированная функция - удобно для подсчета статистик.

Проще показать на примере:

In [201]:
# обычная таблица
select_sql('''
SELECT StudentId, ClassId
FROM StudentClass
''')

[(1, 2), (2, 1), (2, 3), (3, 1), (3, 2), (4, 2)]

In [206]:
# считаем популярность разных классов

select_sql('''
SELECT ClassId, Count(*) as popularity
FROM StudentClass
GROUP BY ClassId
ORDER BY popularity DESC
''')

[(2, 3), (1, 2), (3, 1)]

Попробуйте сами реализовать несколько запросов:

**Выведите для каждого класса его название и популярность среди студентов**.

**Выведите для каждого класса его название и самого старшего студента, который туда ходит**.

**Выведите общее число студентов**.

## Как изучать SQL

Как и у регулярных выражений, у этого языка есть куча сайтов, где его можно изучить в интерактивном режиме.

Для детей: [красивый сайт, простые запросами, простая база, в запросе надо просто заполнить пропуски](https://www.w3schools.com/sql/sql_exercises.asp).

Для подростков: [красивый сайт, запросы усложняются, интересная база, запрос в свободной форме](https://sqlbolt.com/lesson/select_queries_introduction) (рекомендую эту посмотреть сначала).

Для взрослых: [некрасивый сайт, запросы быстро усложняются, скучная база из огромного числа таблиц, запрос в свободной форме](http://www.sql-ex.ru).

## Пишем оболочку над БД на питоне

Не хочется сильно много где в питоновском коде использовать кусочки SQL. Лучше всего - для каждого разного использования базы написать одну функцию, которая будет делать SQL-запрос, и везде в коде использовать именно ее.

In [211]:
# Например, функция, которая выдает всю информацию по человеку по его id-нику.

def get_student_by_id(student_id):
    student = select_sql('''SELECT * FROM Students WHERE StudentId = {}'''.format(student_id))[0]
    return {
        'id': student[0],
        'name': student[1],
        'birth_date': student[2]
    }

In [212]:
get_student_by_id(2)

{'birth_date': datetime.date(2002, 6, 10), 'id': 2, 'name': 'Иван Даниилов'}

Согласитесь, что такими функциями пользоваться гораздо удобнее?

**Напишите функцию, которая возвращает по названию класса список имен всех, кто ходит на него.**

In [213]:
def get_students_by_class_name(class_name):
    pass

In [214]:
get_students_by_class_name('ML')

## SQL-инъекции
![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

В том способе писать оболочки, который я написал выше - есть маленький минус. Вы строите SQL запрос тупо подставив в запрос подстроку. Никто не гарантириует вам, что строка - это честное число. Представьте, что злоумешленнику удалось послать какой-то запрос на сервер и он запустил вот такую функцию:

In [215]:
student_id = '1; DROP TABLE Students;'

get_student_by_id(student_id)

Error in DataBase: no results to fetch


TypeError: 'bool' object is not subscriptable

In [216]:
select_sql('''SELECT * FROM Students''')

Error in DataBase: relation "students" does not exist
LINE 1: SELECT * FROM Students
                      ^



False

Oops...

![](https://pp.userapi.com/c840122/v840122354/8c891/M91k-AjvA6s.jpg)
И да, кто-то в Польше все такие зарегистрировал компанию с названием-SQL-инъекцией.

Есть много способов бороться с SQL-инъекциями:
* не бороться и надеяться на то, что у вас нет места, где вы подставляете в SQL что-то, к чему есть доступ у пользователей = не работает, так как пользователи всегда найдут доступ куда не надо
* в функциях проверять, что student_id - это число, а не строка = прекрасно работает кроме случаев, когда вы подставляете строку (например, имя пользователя)
* строки подставлять внутрь кавычек, запретить строки, содержащие кавычки = с именами нормально, а что делать в тексте с кавычками
* использовать экранирование - тогда даже если внутри имени студента есть тот же тип кавычек, что и у вас, то они не будут считаться кавычками (перед ними появится знак '\' или что-нибудь типа такого).

## P. S. Или все-таки лучше Excel?

![](https://imgs.xkcd.com/comics/spreadsheets.png)